## Calculate the expansion into a vacuum of a plasma of homogeneous temperature


1 - demarrage et conditions initiales

1.1 lecture des donnees

In [1]:
import sys
import numpy as np
import math 
from scipy import special

In [2]:
def Thot(time):
    if time <= 9.9: 
        Thot=1.0
    else:
        Thot=1.0
    return Thot

def Tcold(time):
    if time <= 9.9:
       Tcold=1.0
    else:
       Tcold=1.0
    return Tcold

 
def Gauss(nmax,ncell,A,B,C,F,phi):

# "c   tridiagonal matrix reduction   M*PHI = F
# "       B0 C0	           PHI0     F0
# "      A1 B1 C1              PHI1     F1
# "          .  .  .        *   .    =  .
# "            Ai Bi Ci        PHIi     Fi
# "                .  .  .      .       .
# "                  An Bn     PHIn     Fn
# " 

    Bx=np.empty((nmax),dtype=float)
    Fx=np.empty((nmax),dtype=float)
    j=0
    Bx[j] = B[j]
    Fx[j] = F[j]
    for j in range(1,ncell):
        Bx[j] = B[j] - A[j]*C[j-1]/Bx[j-1] 
        Fx[j] = F[j] - A[j]*Fx[j-1]/Bx[j-1]   

# "C... descending recursion for solution

    j = ncell
    phi[j] = Fx[j]/Bx[j]
        
    for j in range(ncell-1,0,-1):
        phi[j] = ( Fx[j]-C[j]*phi[j+1] )/Bx[j]
            
    return phi[j]

### Input parameters

In [3]:
nmax=150100
njmax=200
ncell=4800 #0; 
nvide=200 #0; 
nlisse=10 #0;
ve_max=7.5; Nbe=76
prog=0.0; itmax=20; tmax=1.; iter0=12; iter1=6;fiter2=0.000
fiter3=0.000; dti=0.2; n0cold=0.0; n0hot=1.; Thmax=1.0; Tcmax=0.001
lfini=True
lmax=50.
nb_cons=True
En_cons=False
EOS=False
T_MeV=1.0
LSS=0.0
nLSS=10.0
nu=0.0
profil='step'
VTS=False
djout=1

In [4]:
#Initialize arrays
# S = numpy.empty((K,K),dtype=float)
dx0 = np.empty((nmax),dtype=float)
qiSS = np.empty((nmax),dtype=float)
dx0  = np.empty((nmax),dtype=float)
x0   = np.empty((nmax),dtype=float)
niSS =np.empty((nmax),dtype=float)
idebut =np.empty((nmax),dtype=float)
irang =np.empty((nmax),dtype=float)
phi = np.empty((nmax),dtype=float)
E = np.empty((nmax),dtype=float)
vemoy = np.empty((nmax),dtype=float)
v = np.empty((nmax),dtype=float)
xt = np.empty((nmax),dtype=float)
v2 = np.empty((nmax),dtype=float)
ni = np.empty((nmax),dtype=float)
gradnh= np.empty((nmax),dtype=float)
gradnc= np.empty((nmax),dtype=float)
ve = np.empty((nmax),dtype=float)
fe0_hot = np.empty((njmax),dtype=float)
fe_hot = np.empty((njmax),dtype=float)
fe0_cold = np.empty((njmax),dtype=float)
fe_cold = np.empty((njmax),dtype=float)
fe = np.empty((njmax),dtype=float)
fe0 = np.empty((njmax),dtype=float)
Te = np.empty((njmax),dtype=float)
dve2s2 = np.empty((njmax),dtype=float)
dve2s2_0 = np.empty((njmax),dtype=float)

Te_hot = np.empty((njmax),dtype=float) 
Te_0 = np.empty((njmax),dtype=float)
Te_cold = np.empty((njmax),dtype=float)
ve2m = np.empty((njmax),dtype=float) 
ve1m = np.empty((njmax),dtype=float)

nhot   = np.empty((nmax),dtype=float)
ncold   = np.empty((nmax),dtype=float)
nhotold = np.empty((nmax),dtype=float)
ncoldold = np.empty((nmax),dtype=float)
phi2old = np.empty((nmax),dtype=float)
phiold = np.empty((nmax),dtype=float)
Eold = np.empty((nmax),dtype=float)
E1s2old = np.empty((nmax),dtype=float)
E1s2= np.empty((nmax),dtype=float)
ghold= np.empty((nmax),dtype=float)
gcold= np.empty((nmax),dtype=float)
xtold= np.empty((nmax),dtype=float)
dxtold= np.empty((nmax),dtype=float)
dxt= np.empty((nmax),dtype=float)
veold= np.empty((njmax),dtype=float)
ve3m = np.empty((njmax),dtype=float)
ve2m = np.empty((njmax),dtype=float)
ve1m = np.empty((njmax),dtype=float)
ni1s2 = np.empty((nmax),dtype=float)
ue = np.empty((njmax),dtype=float)


a=np.empty((nmax),dtype=float)
b=np.empty((nmax),dtype=float)
c=np.empty((nmax-1),dtype=float)
f=np.empty((nmax),dtype=float)

Boundary conditions

In [5]:
Te[Nbe-1]=Thmax
Te_hot[Nbe-1]=Thmax
Te_cold[Nbe-1]=Tcmax

### Initial conditions

In [6]:
if prog < 0.9:
    prog=10.**(-5./ncell)
    
density_profile =['sack', 'step', 'gaus', 'expo'] 

if profil not in density_profile:
        print ('what is the ion density profile?')   
        
if ((not lfini) and (nb_cons or En_cons)):
    nb_cons=False
    En_cons=False
    print('lfini=.false. and then En_cons=.false. and nb_cons=.false.')

if ((not nb_cons) and En_cons):
    nb_cons=True
    print('En_cons=.true.  and then  nb_cons=.true.')

if ((not nb_cons) and (n0cold < 1e-04)):
    nb_cons=False
    print('n0cold < 1e-04  and then  nb_cons=.false.')
      
if ((not lfini) and VTS):
    VTS=False
    print('lfini=.false. and then VTT=.false.')
     
ntotal=ncell+nvide
      
if ntotal > nmax:
    sys.exit('ntotal=ncell+nvide est plus grand que nmax')

omegadt=np.sqrt(n0cold+n0hot)*dti
if (omegadt > 2.0):
    print('No reduced time due to stability')
    dti=2./np.sqrt(n0cold+n0hot)
    print('dti=',dti)


bitemp=False
if Tcmax > 0.0 and n0cold > 0.0:
    bitemp=True

nstep=tmax/dti+0.5
nsort=1
if nstep > 2000:
    nsort=nstep/2000
laststep=False
iline=1
iline_vide=1
if ncell > 3999:
    iline=ncell/2000
if nvide > 799:
    iline_vide=nvide/400     
   

### 1.2 construction du maillage
### 1.2.1 determination de la longueur du plasma

if not lfini:
    time=0.0
    dt=dti
    Th=Thmax
    Tc=Tcmax
    cs2=(n0cold+n0hot)/(n0cold/Tc+n0hot/Th)
    cs=sqrt(cs2)
    if profil=='sack': length=(10.*LSS+25.)*cs
    if profil=='expo': length=(LSS+25.)*cs
    if profil=='step': length=25.*cs   
        
#     if itmax == 1 goto 91
#     if time => (tmax-1.e-5): goto 91
#     if itime ==itmax goto 91

    Th=Thmax*Thot(0.)
    Tc=Tcmax*Tcold(0.)

    for itime in range(1,itmax):
       if (time+dt) > (tmax-1.e-5): dt=tmax-time
       cs2old=cs2
       time=time+dt
       Th=Thmax*Thot(time)
       Tc=Tcmax*Tcold(time)
       cs2=(n0cold+n0hot)/(n0cold/Tc+n0hot/Th)
       cs=0.5*(np.sqrt(cs2old)+np.sqrt(cs2))
       length=length+cs*dt
    
if lfini:
    length=lmax    
    
# 1.2.2 maillage spatial   

rgauss=2.0*length/np.sqrt(np.pi)
ni0=n0cold+n0hot

# Initial step in x

if prog == 1.0:
    dx0[1]=length/ncell
else:
    dx0[1]=length*(1.-prog)/(1.-prog**ncell)

# Periodic conditions

x0[0]=-length
x0[1]=x0[0]+dx0[1]

#Initialize density profiles

for i in [0,1]:
    niSS[i]=ni0
    if profil == 'expo' and x0[i]> -LSS:
         niSS[i]=ni0*np.exp(-(x0[i]+LSS)/LSS)
    if profil =='sack':
         niSS[i]=ni0*(2./np.pi)*math.atan(np.exp(-x0[i]/LSS))
    if profil =='gaus': 
         niSS[i]=ni0*np.exp(-((x0[i]+length)/rgauss)**2)

for i in range(2,ncell):
    if i==2: 
        dx0[2]=(1.0+prog)*dx0[1]*niSS[0]/niSS[1]-dx0[1]
    else:
        dx0[i]=(dx0[i-2]+dx0[i-1])*prog*niSS[i-2]/niSS[i-1]-dx0[i-1]
            
    if profil == 'expo' or profil == 'sack' and dx0[i]> dx0[i-1] and dx0[i] > LSS/5: 
        dx0[i] = LSS/5.
        
    if profil == 'gaus' and dx0[i]> dx0[i-1] and dx0[i]>lmax/5.:
        dx0[i]=lmax/5.0
        
    x0[i]=x0[i-1]+dx0[i]
    niSS[i]=ni0
    if profil == 'expo' and x0[i]> -LSS: 
         niSS[i]=ni0*np.exp(-(x0[i]+LSS)/LSS)
    if profil == 'sack': 
         niSS[i]=ni0*(2.0/np.pi)*math.atan(np.exp(-x0[i]/LSS))
    if profil == 'gaus': 
        niSS[i]=ni0*np.exp(-((x0[i]+length)/rgauss)**2)

qiSS[0]=niSS[0]*dx0[1]

for  i in range(1,ncell-1):
    qiSS[i]=niSS[i]*(dx0[i]+dx0[i+1])/2.0
    qiSS[ncell]=niSS[ncell]*dx0[ncell]*(1.0+prog)/2.
   
# 1.2.3 mise en memoire de l'ordre initial des ions


# idebut(0:nmax),irang(0:nmax)
for  i in range(0,ncell):
    idebut[i]=i
    irang[i]=i
    
    
# 1.3 conditions initiales et estimation du potentiel phi initial

break_=False # pas de deferlement au debut

Th=Thmax*Thot(0.)
Tc=Tcmax*Tcold(0.)
if Th == 0.0:
    sys.exit('la temperature chaude ne doit pas s''annuler')


phi[ncell]=(n0cold*Tc+n0hot*Th)/(n0cold+n0hot)
E[ncell]=np.sqrt(2.*(n0cold*Tc*np.exp(-phi[ncell]/Tc) +n0hot*Th*np.exp(-phi[ncell]/Th)))
alpha=E[ncell]/phi[ncell]


for i in range(0,ncell):
    xt[i]=x0[i]
    v[i]=0.0
    vemoy[i]=0.0
    phi[i]=phi[ncell]*np.exp(alpha*(x0[i]-x0[ncell]))

for i in range(ncell+1,ntotal):
    v[i]=0.0
    vemoy[i]=0.0
    v2[i]=0.0
    ni[i]=1.e-12
    
E[0]=0.0
gradnh[0]=0.0
gradnc[0]=0.0
time=0.0
dt=dti
nhm2=n0hot
nhm1=n0hot
Whot1=0.0
Whot2=0.0
Wcold=0.0
idico=0     

## 1.3.1 initialisation de la fonction de distribution des electrons
## et des temperatures locales

xvide= ve_max**2/np.sqrt(8.0)+0.3
ve_max=ve_max*np.sqrt(Thmax)
fe_min=n0hot/np.sqrt(Thmax)*np.exp(-ve_max**2/Thmax/2.0)

if Thmax ==Tcmax or (not bitemp): 
    prog_v=1.0
    dve=ve_max/(Nbe-1)
else:
    prog_v=(Thmax/Tcmax)**(0.50/(Nbe-2))
    dve=ve_max*(prog_v-1.0)/(prog_v**(Nbe-1)-1.0)


jcoldmax=Nbe
for j in range(0,Nbe-1):
    if j == 0:
        ve[0]=0.0
    else:
        ve[j]=ve[j-1]+dve
        dve=dve*prog_v

    fe0_hot[j]=n0hot/np.sqrt(Thmax)*np.exp(-(ve[j]**2)/Thmax/2.0)
    if(bitemp):
        fe0_cold[j]=n0cold/np.sqrt(Tcmax)*np.exp(-(ve[j]**2)/Tcmax/2.0)
    else:
        fe0_cold[j]=0.0

    if (fe0_cold[j] < fe_min and j < jcoldmax):
        jcoldmax=j

    fe0[j]=fe0_hot[j]+fe0_cold[j]
    fe_hot[j]=fe0_hot[j]
    fe_cold[j]=fe0_cold[j]
    fe[j]=fe0[j]

for  j in range(0,Nbe-2):
          dve2s2=0.50*(ve[j+1]**2-ve[j]**2)
          dve2s2_0[j]=dve2s2
          Te[j]=-dve2s2/(np.log(fe[j+1])-np.log(fe[j]))
          Te_0[j]=Te[j]
          Te_hot[j]= Thmax
          Te_cold[j]= Tcmax

for j in range(0,Nbe-1):
           ve2m[j]=ve[j]
           ve1m[j]=ve[j]

# 2 - boucle temporelle

for itime in range(1,itmax):
    if itime <=  3:
         iter=iter0
    else:
          iter=iter1

# *         2.1 determination ou estimation des temperatures et de n0hot
# *             et calcul de la nouvelle densite ionique
		
# *	2.1.2 - estimation de la modification de n0hot liee a la 
# *             conservation du nombre d'electrons chauds

if (nb_cons):
    nhm3=nhm2
    nhm2=nhm1
    nhm1=n0hot
    n0hot=3.0*nhm1-3.0*nhm2+nhm3    
    
    
    
########## line 478    
if (itime == (itime/nsort)*nsort):
    print('time',time)
    print(Th, Tc, n0hot)
    
    
# *	2.1.4 - mise en memoire des anciennes valeurs de la densite
# *             electronique, du potentiel, du champ electrique,
# *             des gradients de densite electronique,
# *             et de la position dans la gaine vide d'ions

if (itime > 1):
    for i in range(ntotal):
        nhotold[i]=nhot[i] 
        ncoldold[i]=ncold[i] 
        phiold[i]=phi[i] 
        Eold[i]=E[i]
        E1s2old[i+1]=E1s2[i+1] 
        ghold[i]=gradnh[i] 
        gcold[i]=gradnc[i]

    for i in range(ncell+1,ntotal):    
        xtold[i]= xt[i] 
    for i in range(1,ntotal):    
        dxtold[i]= dxt[i] 
        
    Whot1old=Whot1
    Whot2old=Whot2
    Wcoldold=Wcold
    
    for i in range(1,Nbe):
        veold[i]=ve[i]
        
    for i in range(0,ncell):
            phi2old[i]=phiold[i]

    for i in range(ncell+1,ntotal-1):
            phi2old[i]=phiold[i+1]

            
# *	2.1.4 bis - estimation de la modification des vej 

for j in range(0,Nbe-1):
    ve3m[j]=ve2m[j]
    ve2m[j]=ve1m[j]           
    ve1m[j]=ve[j]
    ve[j]=2.0*ve1m[j]-1.0*ve2m[j]
    
    
# *     2.1.5 calcul de la nouvelle densite ionique
for i in range(1,ncell):
       dxt[i]=xt[i]-xt[i-1]
       ni1s2[i]=(qiSS[i-1]+qiSS[i])/dxt[i]/2.
    
ni[0]=qiSS[0]/dxt[1] 

for i in range(1,ncell-1):
    ni[i]=2.0*qiSS[i]/(dxt[i]+dxt[i+1])
   
ni[ncell]=2.0*qiSS[ncell]/dxt[ncell]/(1.0+2.0*dxt[ncell]/dxt[ncell-1]-dxt[ncell-1]/dxt[ncell-2])

for i in range( 1,ncell):
    ni1s2[i]=(ni[i-1]+ni[i])/2.
    
    
# *	   2.2 - iteration pour le calcul de phi, Th, Tc

for iphi in range(1,iter):
    if (itime ==1) :
        for j in range(0,Nbe-1):
            fe_hot[j]=fe_hot[j]*np.exp(-phi[0]/Thmax)
            if(bitemp):fe_cold[j]=fe_cold[j]*np.exp(-phi[0]/Tcmax)
            fe[j]=fe_hot[j]+fe_cold[j]

            
        for i in range(1,ncell):
             phi[i]=phi[i]-phi[0]

        phi[0]=0.0
        for j in range(0,Nbe-2):
            Te_0[j]=-dve2s2_0[j]/(log(fe[j+1])-log(fe[j]))


    for  j in range(0,Nbe-2):
         dve2s2=0.50*(ve[j+1]**2-ve[j]**2)
         cooling=dve2s2/dve2s2_0[j]
         Te[j]=Te_0[j]*cooling
         if(iphi == iter):
            Te_hot[j]=Thmax*cooling
         if(bitemp): 
            Te_cold[j]=Tcmax*cooling
  

    if(itime ==1): 
        phi0 = phi[0]  
        
# c---------------------------------------------------
# c------ Calcul i quelconque c---------------------------------------------------

for i in range(0,ncell):

        nephi=0.0
        gnephi=0.0
        pephi2=0.0
        j=Nbe-2
        k=0  
        
        while ((ve[j]**2-2.0*(phi[i]-phi[0]))>= 0.0 and (j > 0)):

                ue[j+1]=np.sqrt(ve[j+1]**2-2*(phi[i]-phi[0]))
                ue[j]=np.sqrt(ve[j]**2-2*(phi[i]-phi[0]))
    
                nephi=nephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])\
       *(-special.erfc(ue[j+1]/(np.sqrt(2.*Te[j]))) +special.erfc(ue[j]/(np.sqrt(2.0*Te[j]))))*np.sqrt(Te[j]) 

                gnephi=gnephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])\
               *np.sqrt(Te[j])*((-special.erfc(ue[j+1]/(np.sqrt(2.0*Te[j])))
               +special.erfc(ue[j]/(np.sqrt(2.*Te[j])))) *(-1.0/Te[j])\
               -(np.exp(-ue[j+1]**2/(2*Te[j]))/(ue[j+1]/np.sqrt(2.))\
               -np.exp(-ue[j]**2/(2*Te[j]))/(ue[j]/np.sqrt(2.0)))/np.sqrt(np.pi*Te[j]))
            

                if(i == ncell):
                      pephi2=pephi2+fe[j]*np.exp((ve[j]**2/2+phi[0])/Te[j])\
                      *np.sqrt(Te[j])*np.exp(-phi[ncell]/Te[j])*Te[j]\
                      *((-special.erfc(ue[j+1]/np.sqrt(2.0*Te[j]))+special.erfc(ue[j]/np.sqrt(2.0*Te[j])))\
                      -(np.exp(-ue[j+1]**2/(2.0*Te[j]))*ue[j+1]-np.exp(-ue[j]**2/(2.0*Te[j]))*ue[j])
                        *np.sqrt(2.0/(np.pi*Te[j])))
                j=j-1
        
        k=j+1
        if (k >= 1):  
            nephi=nephi+fe[k]*np.exp((ve[k]**2/2+phi[0]-phi[i])/Te[k-1])\
            *(1.0-special.erfc(ue[k]/(np.sqrt(2.0*Te[k-1]))) *np.sqrt(Te[k-1]) )
            
            gnephi=gnephi+fe[k]*np.exp((ve[k]**2/2+phi[0]-phi[i])/Te[k-1])\
            *np.sqrt(Te[k-1])*((-1.0/Te[k-1])*(1.0-special.erfc(ue[k]/np.sqrt(2.0*Te[k-1])))\
            -(np.exp(-ue[k]**2/(2*Te[k-1]))/(ue[k]/np.sqrt(2.0)))/np.sqrt(np.pi*Te[k-1]))
            
            if(i == ncell):
               pephi2=pephi2+fe[k]*np.exp((ve[k]**2/2+phi[0])/Te[k-1])*np.sqrt(Te[k-1])\
               *np.exp(-phi[ncell]/Te[k-1])*Te[k-1]*(1.0-special.erfc(ue[k]/np.sqrt(2.0*Te[k-1]))\
               -np.exp(-ue[k]**2/(2.0*Te[k-1]))*ue[k]*np.sqrt(2.0/(np.pi*Te[k-1]))) 

        j=Nbe-1
        nephi=nephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])\
        *special.erfc(ue[j]/np.sqrt(2.0*Te[j]))*np.sqrt(Te[j]) 

        gnephi=gnephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])*np.sqrt(Te[j])\
        *(special.erfc(ue[j]/np.sqrt(2.0*Te[j]))*(-1./Te[j])\
        -(0.0 -np.exp(-ue[j]**2/(2*Te[j]))/(ue[j]/np.sqrt(2.0)))/np.sqrt(np.pi*Te[j]))

        if(i==ncell):
            pephi2=pephi2+fe[j]*exp((ve[j]**2/2+phi[0])/Te[j])*np.sqrt(Te[j])*np.exp(-phi[ncell]/Te[j])\
            *Te[j]*((special.erfc(ue[j]/np.sqrt(2.*Te[j])))\
            -(0.-np.exp(-ue[j]**2/(2.0*Te[j]))*ue[j])*np.sqrt(2.0/(np.pi*Te[j])))  
            pephi=np.sqrt(2.0*pephi2)
            gpephi=-nephi/pephi
                
       
        if(i == 0):
           b[0]=-2.*dxt[1]**2+gnephi
           c[0]=2.0/dxt[1]**2
           f[0]=ni[0]-nephi+phi[0]*gnephi
        elif (i > 0 and i < ncell):
            
           a[i]=2.0/dxt[i]/(dxt[i]+dxt[i+1])
           b[i]=-2.0/(dxt[i]*dxt[i+1])+gnephi
           c[i]=2.0/dxt[i+1]/(dxt[i]+dxt[i+1])
           f[i]=ni[i]-nephi+phi[i]*gnephi  
        elif (i == ncell):
           a[ncell]=2.0/dxt[ncell]**2
           b[ncell]=-a[ncell] +gpephi*2.0/dxt[ncell] +gnephi
           f[ncell]=ni[ncell]-(nephi-phi[ncell]*gnephi)-(pephi-phi[ncell]*gpephi)*2./dxt[ncell]
        
        
       
# *	2.2.2 - inversion de la matrice tridiagonale
 
Gauss(nmax,ncell,a,b,c,f,phi)

time 0.0
1.0 0.001 1.0


<ipython-input-6-c5f059249068>:322: RuntimeWarning: invalid value encountered in double_scalars
  ni[ncell]=2.0*qiSS[ncell]/dxt[ncell]/(1.0+2.0*dxt[ncell]/dxt[ncell-1]-dxt[ncell-1]/dxt[ncell-2])
<ipython-input-6-c5f059249068>:375: RuntimeWarning: divide by zero encountered in double_scalars
  nephi=nephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])\
<ipython-input-6-c5f059249068>:376: RuntimeWarning: invalid value encountered in double_scalars
  *(-special.erfc(ue[j+1]/(np.sqrt(2.*Te[j]))) +special.erfc(ue[j]/(np.sqrt(2.0*Te[j]))))*np.sqrt(Te[j])
<ipython-input-6-c5f059249068>:376: RuntimeWarning: divide by zero encountered in double_scalars
  *(-special.erfc(ue[j+1]/(np.sqrt(2.*Te[j]))) +special.erfc(ue[j]/(np.sqrt(2.0*Te[j]))))*np.sqrt(Te[j])
<ipython-input-6-c5f059249068>:378: RuntimeWarning: divide by zero encountered in double_scalars
  gnephi=gnephi+fe[j]*np.exp((ve[j]**2/2+phi[0]-phi[i])/Te[j])\
<ipython-input-6-c5f059249068>:378: RuntimeWarning: invalid value encountered in d

nan